# Analiza czerniaka za pomocą fraktalnej sieci neuronowej

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Sprawdzamy dostępne urządzenie

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Zapisujemy konfigurację do zmiennych.

In [4]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

Definiujemy warstwę, która będzie tworzyła obraz z fraktalnych cech podanych jej obrazów.

In [5]:
class Fractal2D(tf.keras.layers.Layer):
    def __init__(self, kernel_size_range):
        super(Fractal2D, self).__init__()
        self.kernel_size_range = kernel_size_range
    
    @staticmethod
    def binarize(patched_image_batch, kernel_size):
        center = kernel_size // 2 + 1
        auxiliary_batch = np.array([[np.full((kernel_size, kernel_size, 3), patch[center][center]) 
                   for patch in patched_image] 
                  for patched_image in patched_image_batch])
        print(f"\nauxiliary batch: {auxiliary_batch.shape}")
        
        ch_binary = tf.cast(tf.math.less_equal(tf.math.abs(tf.math.subtract(patched_image_batch, auxiliary_batch)), kernel_size), tf.int32)
        print(f"\ch_binary batch: {ch_binary.shape}")

    def call(self, image_batch):
        print(f"\nimage batch shape: {image_batch.shape}")
        for kernel_size in range(self.kernel_size_range[0], self.kernel_size_range[1] + 1, 2):
            patched_image_batch = tf.image.extract_patches(image_batch,
                                                         sizes=(1, kernel_size, kernel_size, 1),
                                                         strides=(1, kernel_size, kernel_size, 1),
                                                         rates=(1, 1, 1, 1),
                                                         padding='SAME')
            batch_size, row_size, col_size, depth = patched_image_batch.shape
            patched_image_batch = tf.reshape(patched_image_batch, shape=(batch_size, row_size * col_size, kernel_size, kernel_size, 3))

            self.binarize(patched_image_batch, kernel_size)
        return image_batch

Ładujemy dane do trenowania i walidacji.

In [6]:
datagen = ImageDataGenerator(validation_split=0.2, rescale=1.0)
training_set = datagen.flow_from_directory('/small-data',
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           batch_size=BATCH_SIZE,
                                           class_mode='categorical',
                                           subset='training')
validation_set = datagen.flow_from_directory('/small-data',
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=BATCH_SIZE,
                                             class_mode='categorical',
                                             subset='validation')

Found 640 images belonging to 3 classes.
Found 159 images belonging to 3 classes.


Zapisujemy ilość rozpoznawalnych diagnoz.

In [7]:
DIAGNOSIS_NUMBER = len(training_set.class_indices)

Tworzymy model, który wykorzystuje wcześniej zdefiniowaną warstwę.

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3), batch_size=32),
    Fractal2D(kernel_size_range=(3, 41)),
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", output_shape=[1280],
                   trainable=False),
    tf.keras.layers.Dense(DIAGNOSIS_NUMBER, activation='softmax')
])


image batch shape: (32, 224, 224, 3)


OperatorNotAllowedInGraphError: Exception encountered when calling layer "fractal2d" (type Fractal2D).

in user code:

    File "<ipython-input-5-25796d5a5e6c>", line 28, in call  *
        self.binarize(patched_image_batch, kernel_size)
    File "<ipython-input-5-25796d5a5e6c>", line 9, in binarize  *
        auxiliary_batch = np.array([[np.full((kernel_size, kernel_size, 3), patch[center][center])

    OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.


Call arguments received:
  • image_batch=tf.Tensor(shape=(32, 224, 224, 3), dtype=float32)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(training_set, validation_data=validation_set, epochs=20)